In [2]:
from yarl import URL
import requests
from bs4 import BeautifulSoup

In [3]:
def create_session() -> requests.Session:
    """`create_session`: Create a requests session with a custom header to make the server believe the scraper is actually a browser opened by a real user.

    `Returns`
    --------- ::

        requests.Session

    `Example(s)`
    ---------

    >>> create_session()
    ... <requests.sessions.Session at 0x223e7834df0>"""
    session = requests.Session()
    session.headers.update(
        {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"
        }
    )
    return session


In [32]:
URL_INIT = URL.build(scheme="https", host="vinatis.com")

In [34]:
URL_INIT

URL('https://vinatis.com')

- Idée : on fait varier le $i$ des pages de 2 à 97, et le query string 

In [35]:
WHITE = "achat-vin-blanc"
RED = "achat-vin-rouge"
ROSE = "achat-vin-rose"

In [36]:
URL_INIT / RED % {"page": 2, "tri":7}

URL('https://vinatis.com/achat-vin-rouge?page=2&tri=7')